In [1]:
from LSTM import LSTM
import os
import pandas as pd
import math
import DataPreparationLSTM
import tensorflow as tf

2024-05-11 17:10:36.358643: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_data = pd.read_csv('data/SPY_data_5min.csv')
input_data['time'] = pd.to_datetime(input_data['time'])

# Filter trades within the time window from 09:30 to 16:00
input_data = input_data[(input_data['time'].dt.time >= pd.to_datetime('09:30').time()) & 
                 (input_data['time'].dt.time <= pd.to_datetime('16:00').time())]

input_data.drop(['Unnamed: 0','money','open','high','low'], axis=1, inplace=True)
input_data['time'] = pd.to_datetime(input_data['time']).dt.strftime('%H:%M')


In [3]:
#Defining features
futures = [1,5]
models = []

models.append([1,20,False,0.01,20,20,0,0]) # future, lag, semi_variance, learning_rate, layer_1, layer_2, layer_3, layer_4
models.append([1, 20, True, 0.01, 40, 40, 0, 0])
models.append([1, 30, False, 0.01, 20, 20, 0, 0])
models.append([1, 30, True, 0.01, 20, 20, 0, 0])
models.append([5, 30, False, 0.01, 20, 40, 0, 0])
models.append([5, 30, True, 0.01, 20, 20, 0, 0])

for model in models:
    print(model[0])

1
1
1
1
5
5


In [4]:
results = {}

for m in models:

    prepared_data = DataPreparationLSTM.DataPreparationLSTM(

                df = input_data,

                future = m[0],

                lag = m[1],

                min_max_scaler=True,

                log_transform=True,

                semi_variance=m[2],

                jump_detect=True,

                period_train= list(
                        [
                            pd.to_datetime("2006-01-01",format = "%Y-%m-%d"),
                            pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),

                        ]
),

                period_validation = list(
                        [

                        pd.to_datetime("2013-01-01",format = "%Y-%m-%d"),
                        pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  

                                ]

                ),

                period_test= list(
                        [
                            pd.to_datetime("2016-01-01",format = "%Y-%m-%d"), 
                            pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),

                        ]
                    ),
)


    prepared_data.prepare_all()
    
    tf.keras.backend.clear_session()

    
    model = LSTM(
                train_matrix = prepared_data.x_train,
                train_y = prepared_data.y_train,
                test_matrix= prepared_data.x_test,
                test_y = prepared_data.y_test,
                epochs = 50,
                learning_rate = m[3],
                layer_one= m[4],
                layer_two= m[5],
                layer_three=m[6],
                layer_four= m[7]
            )


    model.train_lstm()
    model.predict_lstm()
            
    prediction_train = model.prediction_train
    prediction_test = model.prediction_test        
    target_scaler = prepared_data.applied_scaler_targets
        
    prediction_train_normal = prepared_data.back_transformation(prediction_train)
    prediction_test_normal = prepared_data.back_transformation(prediction_test)
        
    train_y_normal = prepared_data.back_transformation(model.train_y)
    test_y_normal = prepared_data.back_transformation(model.test_y)
        
    model.prediction_train = prediction_train_normal
    model.prediction_test = prediction_test_normal
    model.test_y = test_y_normal
    model.train_y = train_y_normal
    
    model.make_accuracy_measures()

    results[f'{m[0]}_{m[1]}_{m[2]}'] = [model.train_accuracy, model.test_accuracy]

2024-05-11 17:10:40.727984: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/50
77/77 [==============================] - 4s 27ms/step - loss: 0.0077 - val_loss: 0.0056
Epoch 2/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0042 - val_loss: 0.0047
Epoch 3/50
77/77 [==============================] - 2s 22ms/step - loss: 0.0055 - val_loss: 0.0066
Epoch 4/50
77/77 [==============================] - 1s 15ms/step - loss: 0.0046 - val_loss: 0.0067
Epoch 5/50
77/77 [==============================] - 1s 13ms/step - loss: 0.0041 - val_loss: 0.0053
Epoch 6/50
77/77 [==============================] - 1s 15ms/step - loss: 0.0040 - val_loss: 0.0052
Epoch 7/50
77/77 [==============================] - 1s 14ms/step - loss: 0.0037 - val_loss: 0.0057
Epoch 8/50
77/77 [==============================] - 1s 12ms/step - loss: 0.0033 - val_loss: 0.0045
Epoch 9/50
77/77 [==============================] - 1s 16ms/step - loss: 0.0033 - val_loss: 0.0039
Epoch 10/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0034 - val_loss: 0.0044
Epoch 11/

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/DataPreparationLSTM.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 5s 41ms/step - loss: 0.0079 - val_loss: 0.0081
Epoch 2/50
77/77 [==============================] - 2s 31ms/step - loss: 0.0055 - val_loss: 0.0102
Epoch 3/50
77/77 [==============================] - 3s 42ms/step - loss: 0.0043 - val_loss: 0.0077
Epoch 4/50
77/77 [==============================] - 3s 37ms/step - loss: 0.0045 - val_loss: 0.0078
Epoch 5/50
77/77 [==============================] - 3s 44ms/step - loss: 0.0043 - val_loss: 0.0086
Epoch 6/50
77/77 [==============================] - 4s 52ms/step - loss: 0.0039 - val_loss: 0.0103
Epoch 7/50
77/77 [==============================] - 3s 39ms/step - loss: 0.0037 - val_loss: 0.0076
Epoch 8/50
77/77 [==============================] - 3s 35ms/step - loss: 0.0038 - val_loss: 0.0067
Epoch 9/50
77/77 [==============================] - 3s 39ms/step - loss: 0.0035 - val_loss: 0.0073
Epoch 10/50
77/77 [==============================] - 2s 32ms/step - loss: 0.0035 - val_loss: 0.0071
Epoch 11/

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/DataPreparationLSTM.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 4s 36ms/step - loss: 0.0073 - val_loss: 0.0067
Epoch 2/50
77/77 [==============================] - 3s 33ms/step - loss: 0.0048 - val_loss: 0.0066
Epoch 3/50
77/77 [==============================] - 2s 28ms/step - loss: 0.0037 - val_loss: 0.0046
Epoch 4/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0031 - val_loss: 0.0042
Epoch 5/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0031 - val_loss: 0.0049
Epoch 6/50
77/77 [==============================] - 2s 26ms/step - loss: 0.0030 - val_loss: 0.0050
Epoch 7/50
77/77 [==============================] - 2s 28ms/step - loss: 0.0029 - val_loss: 0.0050
Epoch 8/50
77/77 [==============================] - 2s 28ms/step - loss: 0.0030 - val_loss: 0.0062
Epoch 9/50
77/77 [==============================] - 2s 24ms/step - loss: 0.0031 - val_loss: 0.0044
Epoch 10/50
77/77 [==============================] - 2s 27ms/step - loss: 0.0032 - val_loss: 0.0083
Epoch 11/

In [10]:

additional_models = []
additional_models.append([5,20,True, 0.01, 20, 40, 0, 0])
additional_models.append([5,20,False, 0.01, 20, 40, 0, 0])
additional_models.append([20,20, True, 0.01, 40, 20, 0, 0])
additional_models.append([20, 20, False, 0.01, 40, 20, 0, 0])

for m in additional_models:

    prepared_data = DataPreparationLSTM.DataPreparationLSTM(

                df = input_data,

                future = m[0],

                lag = m[1],

                min_max_scaler=True,

                log_transform=True,

                semi_variance=m[2],

                jump_detect=True,

                period_train= list(
                        [
                            pd.to_datetime("2006-01-01",format = "%Y-%m-%d"),
                            pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),

                        ]
),

                period_validation = list(
                        [

                        pd.to_datetime("2013-01-01",format = "%Y-%m-%d"),
                        pd.to_datetime("2015-12-31",format = "%Y-%m-%d"),  

                                ]

                ),

                period_test= list(
                        [
                            pd.to_datetime("2016-01-01",format = "%Y-%m-%d"), 
                            pd.to_datetime("2018-12-31",format = "%Y-%m-%d"),

                        ]
                    ),
)


    prepared_data.prepare_all()
    
    tf.keras.backend.clear_session()

    
    model = LSTM(
                train_matrix = prepared_data.x_train,
                train_y = prepared_data.y_train,
                test_matrix= prepared_data.x_test,
                test_y = prepared_data.y_test,
                epochs = 50,
                learning_rate = m[3],
                layer_one= m[4],
                layer_two= m[5],
                layer_three=m[6],
                layer_four= m[7]
            )


    model.train_lstm()
    model.predict_lstm()
            
    prediction_train = model.prediction_train
    prediction_test = model.prediction_test        
    target_scaler = prepared_data.applied_scaler_targets
        
    prediction_train_normal = prepared_data.back_transformation(prediction_train)
    prediction_test_normal = prepared_data.back_transformation(prediction_test)
        
    train_y_normal = prepared_data.back_transformation(model.train_y)
    test_y_normal = prepared_data.back_transformation(model.test_y)
        
    model.prediction_train = prediction_train_normal
    model.prediction_test = prediction_test_normal
    model.test_y = test_y_normal
    model.train_y = train_y_normal
    
    model.make_accuracy_measures()

    results[f'{m[0]}_{m[1]}_{m[2]}'] = [model.train_accuracy, model.test_accuracy]

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/DataPreparationLSTM.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 3s 21ms/step - loss: 0.0107 - val_loss: 0.0064
Epoch 2/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0047 - val_loss: 0.0067
Epoch 3/50
77/77 [==============================] - 2s 27ms/step - loss: 0.0042 - val_loss: 0.0056
Epoch 4/50
77/77 [==============================] - 1s 19ms/step - loss: 0.0034 - val_loss: 0.0059
Epoch 5/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0033 - val_loss: 0.0071
Epoch 6/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0032 - val_loss: 0.0058
Epoch 7/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0032 - val_loss: 0.0056
Epoch 8/50
77/77 [==============================] - 2s 29ms/step - loss: 0.0031 - val_loss: 0.0051
Epoch 9/50
77/77 [==============================] - 2s 30ms/step - loss: 0.0031 - val_loss: 0.0062
Epoch 10/50
77/77 [==============================] - 2s 20ms/step - loss: 0.0032 - val_loss: 0.0061
Epoch 11/

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/DataPreparationLSTM.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 3s 23ms/step - loss: 0.0109 - val_loss: 0.0075
Epoch 2/50
77/77 [==============================] - 2s 27ms/step - loss: 0.0046 - val_loss: 0.0053
Epoch 3/50
77/77 [==============================] - 2s 27ms/step - loss: 0.0038 - val_loss: 0.0066
Epoch 4/50
77/77 [==============================] - 1s 19ms/step - loss: 0.0040 - val_loss: 0.0085
Epoch 5/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0040 - val_loss: 0.0051
Epoch 6/50
77/77 [==============================] - 1s 17ms/step - loss: 0.0035 - val_loss: 0.0062
Epoch 7/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0034 - val_loss: 0.0049
Epoch 8/50
77/77 [==============================] - 2s 26ms/step - loss: 0.0033 - val_loss: 0.0058
Epoch 9/50
77/77 [==============================] - 2s 28ms/step - loss: 0.0034 - val_loss: 0.0058
Epoch 10/50
77/77 [==============================] - 2s 26ms/step - loss: 0.0036 - val_loss: 0.0060
Epoch 11/

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/DataPreparationLSTM.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 4s 34ms/step - loss: 0.0120 - val_loss: 0.0086
Epoch 2/50
77/77 [==============================] - 2s 24ms/step - loss: 0.0075 - val_loss: 0.0082
Epoch 3/50
77/77 [==============================] - 2s 25ms/step - loss: 0.0071 - val_loss: 0.0110
Epoch 4/50
77/77 [==============================] - 2s 25ms/step - loss: 0.0070 - val_loss: 0.0095
Epoch 5/50
77/77 [==============================] - 1s 17ms/step - loss: 0.0057 - val_loss: 0.0090
Epoch 6/50
77/77 [==============================] - 1s 19ms/step - loss: 0.0057 - val_loss: 0.0087
Epoch 7/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0056 - val_loss: 0.0099
Epoch 8/50
77/77 [==============================] - 2s 29ms/step - loss: 0.0056 - val_loss: 0.0093
Epoch 9/50
77/77 [==============================] - 2s 22ms/step - loss: 0.0056 - val_loss: 0.0118
Epoch 10/50
77/77 [==============================] - 2s 21ms/step - loss: 0.0058 - val_loss: 0.0104
Epoch 11/

/home/misikori/Desktop/Projects/VolatilityPredictionMatf/DataPreparationLSTM.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output["Target"] = help_df.mean(axis=1)


Epoch 1/50
77/77 [==============================] - 4s 27ms/step - loss: 0.0096 - val_loss: 0.0135
Epoch 2/50
77/77 [==============================] - 1s 19ms/step - loss: 0.0067 - val_loss: 0.0135
Epoch 3/50
77/77 [==============================] - 2s 23ms/step - loss: 0.0059 - val_loss: 0.0100
Epoch 4/50
77/77 [==============================] - 1s 18ms/step - loss: 0.0062 - val_loss: 0.0152
Epoch 5/50
77/77 [==============================] - 2s 22ms/step - loss: 0.0060 - val_loss: 0.0116
Epoch 6/50
77/77 [==============================] - 2s 28ms/step - loss: 0.0066 - val_loss: 0.0104
Epoch 7/50
77/77 [==============================] - 2s 24ms/step - loss: 0.0063 - val_loss: 0.0111
Epoch 8/50
77/77 [==============================] - 2s 27ms/step - loss: 0.0057 - val_loss: 0.0135
Epoch 9/50
77/77 [==============================] - 2s 25ms/step - loss: 0.0058 - val_loss: 0.0087
Epoch 10/50
77/77 [==============================] - 2s 24ms/step - loss: 0.0060 - val_loss: 0.0131
Epoch 11/

In [11]:
results

{'1_20_False': [{'MSE': 9.548034200874695e-09,
   'MAE': 3.235962318363265e-05,
   'RSquared': 0.7176906712394608},
  {'MSE': 1.0966375058596514e-09,
   'MAE': 1.6509384345671718e-05,
   'RSquared': 0.5912243272869835}],
 '1_20_True': [{'MSE': 1.3124395502084137e-08,
   'MAE': 3.401024488886869e-05,
   'RSquared': 0.611947422199034},
  {'MSE': 1.084951953934015e-09,
   'MAE': 1.7497384811002865e-05,
   'RSquared': 0.5955801598423186}],
 '1_30_False': [{'MSE': 1.685690810463549e-08,
   'MAE': 5.1502033806766844e-05,
   'RSquared': 0.5015872050854427},
  {'MSE': 1.7602957253276232e-09,
   'MAE': 2.0558437385114596e-05,
   'RSquared': 0.3438432796162846}],
 '1_30_True': [{'MSE': 1.2462328216400942e-08,
   'MAE': 3.7944007615713705e-05,
   'RSquared': 0.6315229460276365},
  {'MSE': 1.1017642801501217e-09,
   'MAE': 1.9228136502651074e-05,
   'RSquared': 0.5893133032719952}],
 '5_30_False': [{'MSE': 6.708318888286331e-09,
   'MAE': 3.0413097122403725e-05,
   'RSquared': 0.75787182912734},
 

In [16]:
for key,value in results.items():
    print(key)
    print(value)
    

1_20_False
[{'MSE': 9.548034200874695e-09, 'MAE': 3.235962318363265e-05, 'RSquared': 0.7176906712394608}, {'MSE': 1.0966375058596514e-09, 'MAE': 1.6509384345671718e-05, 'RSquared': 0.5912243272869835}]
1_20_True
[{'MSE': 1.3124395502084137e-08, 'MAE': 3.401024488886869e-05, 'RSquared': 0.611947422199034}, {'MSE': 1.084951953934015e-09, 'MAE': 1.7497384811002865e-05, 'RSquared': 0.5955801598423186}]
1_30_False
[{'MSE': 1.685690810463549e-08, 'MAE': 5.1502033806766844e-05, 'RSquared': 0.5015872050854427}, {'MSE': 1.7602957253276232e-09, 'MAE': 2.0558437385114596e-05, 'RSquared': 0.3438432796162846}]
1_30_True
[{'MSE': 1.2462328216400942e-08, 'MAE': 3.7944007615713705e-05, 'RSquared': 0.6315229460276365}, {'MSE': 1.1017642801501217e-09, 'MAE': 1.9228136502651074e-05, 'RSquared': 0.5893133032719952}]
5_30_False
[{'MSE': 6.708318888286331e-09, 'MAE': 3.0413097122403725e-05, 'RSquared': 0.75787182912734}, {'MSE': 8.548969334412093e-10, 'MAE': 1.9365494918123494e-05, 'RSquared': 0.58731906495

In [21]:
    output_directory = "output/LSTM_outputdata"
    os.makedirs(output_directory,exist_ok=True)
    for name, value in results.items():
        output_file_path = os.path.join(output_directory,f'{name}.txt')
          
        with open(output_file_path, 'w') as output_file:
            output_file.write("Train Accuracy  \n")
            output_file.write(f"{value[0]} \n")
            output_file.write("Test Accuracy  \n")
            output_file.write(f"{value[1]} \n ")
